In [23]:
from aequitas.plotting import Plot
ap = Plot()
import pandas as pd

import aequitas.plot as aq
from aequitas.group import Group
from aequitas.bias import Bias 
from aequitas.fairness import Fairness

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, label_binarize, LabelBinarizer
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [34]:
import sys

sys.path.insert(0, "../elbaite/ml")

In [21]:

from data import process_data
from model import train_model, compute_model_metrics, inference

In [25]:
data = pd.read_csv("../data/clean_census.csv")
data.head()

,age,workclass,fnlgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [26]:
train, test = train_test_split(data, test_size=0.20)


In [68]:
data_aq = test.drop(columns=["salary"], axis=1)

In [69]:
data_aq.head()

,age,workclass,fnlgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
13308,45,Private,88500,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,44,United-States
18956,21,Private,399022,HS-grad,9,Never-married,Farming-fishing,Own-child,White,Male,0,0,24,United-States
19656,33,Private,171215,Masters,14,Never-married,Adm-clerical,Own-child,White,Male,0,0,40,United-States
30349,46,Private,153254,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,0,0,40,United-States
10624,52,Private,185407,Assoc-voc,11,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,48,Poland


In [28]:
cat_features = [
        "workclass",
        "education",
        "marital-status",
        "occupation",
        "relationship",
        "race",
        "sex",
        "native-country",
    ]

In [29]:
X_train, y_train, encoder, lb = process_data(
        train, categorical_features=cat_features, label="salary", training=True
    )

In [32]:
X_test, y_test, *_ = process_data(
        test, categorical_features=cat_features, label="salary", training=False, encoder=encoder, lb=lb
    )

In [30]:
model = train_model(X_train, y_train)

In [33]:
pred = inference(model, X_test)
scores = model.predict_proba(X_test)

precision, recall, fbeta = compute_model_metrics(y_test, pred)
print(f"Precision: {precision}")
print(f"recall: {recall}")
print(f"fbeta: {fbeta}")

Precision: 0.5433163465307722
recall: 0.8755527479469362
fbeta: 0.6705370101596517


In [71]:
data_aq["label_value"] = y_test
data_aq["score"] = pred
data_aq.head()

,age,workclass,fnlgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label_value,score
13308,45,Private,88500,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,44,United-States,1,1
18956,21,Private,399022,HS-grad,9,Never-married,Farming-fishing,Own-child,White,Male,0,0,24,United-States,0,0
19656,33,Private,171215,Masters,14,Never-married,Adm-clerical,Own-child,White,Male,0,0,40,United-States,0,0
30349,46,Private,153254,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,0,0,40,United-States,0,1
10624,52,Private,185407,Assoc-voc,11,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,48,Poland,1,1


In [40]:
data_aq.dtypes

age               object
workclass         object
fnlgt             object
education         object
education-num     object
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain      object
capital-loss      object
hours-per-week    object
native-country    object
label_value        int64
scores             int64
dtype: object

In [70]:
for col in data_aq.columns:
    data_aq[col] = data_aq[col].astype(str)

In [72]:
group = Group()
xtab, idxs = group.get_crosstabs(data_aq)
xtab.head()

,model_id,score_threshold,k,attribute_name,attribute_value,tpr,tnr,for,fdr,fpr,fnr,npv,precision,pp,pn,ppr,pprev,fp,fn,tn,tp,group_label_pos,group_label_neg,group_size,total_entities,prev
0,0,binary 0/1,2640,age,17,NaN,1.0,0.000000,NaN,0.0,NaN,1.000000,NaN,0,76,0.0,0.0,0,0,76,0,0,76,76,6513,0.000000
1,0,binary 0/1,2640,age,18,NaN,1.0,0.000000,NaN,0.0,NaN,1.000000,NaN,0,108,0.0,0.0,0,0,108,0,0,108,108,6513,0.000000
2,0,binary 0/1,2640,age,19,0.0,1.0,0.014286,NaN,0.0,1.0,0.985714,NaN,0,140,0.0,0.0,0,2,138,0,2,138,140,6513,0.014286
3,0,binary 0/1,2640,age,20,NaN,1.0,0.000000,NaN,0.0,NaN,1.000000,NaN,0,151,0.0,0.0,0,0,151,0,0,151,151,6513,0.000000
4,0,binary 0/1,2640,age,21,0.0,1.0,0.014706,NaN,0.0,1.0,0.985294,NaN,0,136,0.0,0.0,0,2,134,0,2,134,136,6513,0.014706


In [73]:
bias = Bias()
bias_df = bias.get_disparity_major_group(xtab, original_df=data_aq, alpha=0.05, mask_significance=True)
bias_df.head()

get_disparity_major_group()


,model_id,score_threshold,k,attribute_name,attribute_value,tpr,tnr,for,fdr,fpr,fnr,npv,precision,pp,pn,ppr,pprev,fp,fn,tn,tp,group_label_pos,group_label_neg,group_size,total_entities,prev,ppr_disparity,pprev_disparity,precision_disparity,fdr_disparity,for_disparity,fpr_disparity,fnr_disparity,tpr_disparity,tnr_disparity,npv_disparity,ppr_ref_group_value,pprev_ref_group_value,precision_ref_group_value,fdr_ref_group_value,for_ref_group_value,fpr_ref_group_value,fnr_ref_group_value,tpr_ref_group_value,tnr_ref_group_value,npv_ref_group_value
0,0,binary 0/1,2640,age,17,NaN,1.0,0.000000,NaN,0.0,NaN,1.000000,NaN,0,76,0.0,0.0,0,0,76,0,0,76,76,6513,0.000000,0.0,0.0,NaN,NaN,0.000000,0.0,NaN,NaN,1.613636,1.125000,35,35,35,35,35,35,35,35,35,35
1,0,binary 0/1,2640,age,18,NaN,1.0,0.000000,NaN,0.0,NaN,1.000000,NaN,0,108,0.0,0.0,0,0,108,0,0,108,108,6513,0.000000,0.0,0.0,NaN,NaN,0.000000,0.0,NaN,NaN,1.613636,1.125000,35,35,35,35,35,35,35,35,35,35
2,0,binary 0/1,2640,age,19,0.0,1.0,0.014286,NaN,0.0,1.0,0.985714,NaN,0,140,0.0,0.0,0,2,138,0,2,138,140,6513,0.014286,0.0,0.0,NaN,NaN,0.128571,0.0,3.818182,0.0,1.613636,1.108929,35,35,35,35,35,35,35,35,35,35
3,0,binary 0/1,2640,age,20,NaN,1.0,0.000000,NaN,0.0,NaN,1.000000,NaN,0,151,0.0,0.0,0,0,151,0,0,151,151,6513,0.000000,0.0,0.0,NaN,NaN,0.000000,0.0,NaN,NaN,1.613636,1.125000,35,35,35,35,35,35,35,35,35,35
4,0,binary 0/1,2640,age,21,0.0,1.0,0.014706,NaN,0.0,1.0,0.985294,NaN,0,136,0.0,0.0,0,2,134,0,2,134,136,6513,0.014706,0.0,0.0,NaN,NaN,0.132353,0.0,3.818182,0.0,1.613636,1.108456,35,35,35,35,35,35,35,35,35,35


In [74]:
fairness = Fairness()
fairness_df = fairness.get_group_value_fairness(bias_df)
fairness_df.tail()

,model_id,score_threshold,k,attribute_name,attribute_value,tpr,tnr,for,fdr,fpr,fnr,npv,precision,pp,pn,ppr,pprev,fp,fn,tn,tp,group_label_pos,group_label_neg,group_size,total_entities,prev,ppr_disparity,pprev_disparity,precision_disparity,fdr_disparity,for_disparity,fpr_disparity,fnr_disparity,tpr_disparity,tnr_disparity,npv_disparity,ppr_ref_group_value,pprev_ref_group_value,precision_ref_group_value,fdr_ref_group_value,for_ref_group_value,fpr_ref_group_value,fnr_ref_group_value,tpr_ref_group_value,tnr_ref_group_value,npv_ref_group_value,Statistical Parity,Impact Parity,FDR Parity,FPR Parity,FOR Parity,FNR Parity,TPR Parity,TNR Parity,NPV Parity,Precision Parity,TypeI Parity,TypeII Parity,Equalized Odds,Unsupervised Fairness,Supervised Fairness
6282,0,binary 0/1,2640,native-country,Thailand,NaN,1.000000,0.000000,NaN,0.000000,NaN,1.000000,NaN,0,1,0.000000,0.000000,0,0,1,0,0,1,1,6513,0.000000,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,1.330467,1.046069,United-States,United-States,United-States,United-States,United-States,United-States,United-States,United-States,United-States,United-States,False,False,NaN,False,False,NaN,NaN,False,True,NaN,False,False,False,False,False
6283,0,binary 0/1,2640,native-country,Trinadad&Tobago,0.000000,0.666667,0.200000,1.000000,0.333333,1.000000,0.800000,0.000000,2,5,0.000758,0.285714,2,1,4,0,1,6,7,6513,0.142857,0.000834,0.691698,0.000000,2.227695,4.541333,1.342007,9.806667,0.000000,0.886978,0.836855,United-States,United-States,United-States,United-States,United-States,United-States,United-States,United-States,United-States,United-States,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False
6284,0,binary 0/1,2640,native-country,United-States,0.898029,0.751616,0.044040,0.448894,0.248384,0.101971,0.955960,0.551106,2397,3406,0.907955,0.413062,1076,150,3256,1321,1471,4332,5803,6513,0.253490,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,United-States,United-States,United-States,United-States,United-States,United-States,United-States,United-States,United-States,United-States,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
6285,0,binary 0/1,2640,native-country,Vietnam,0.666667,0.666667,0.090909,0.714286,0.333333,0.333333,0.909091,0.285714,7,11,0.002652,0.388889,5,1,10,2,3,15,18,6513,0.166667,0.002920,0.941478,0.518438,1.591211,2.064242,1.342007,3.268889,0.742367,0.886978,0.950972,United-States,United-States,United-States,United-States,United-States,United-States,United-States,United-States,United-States,United-States,False,True,False,False,False,False,False,True,True,False,False,False,False,False,False
6286,0,binary 0/1,2640,native-country,Yugoslavia,0.500000,1.000000,0.333333,0.000000,0.000000,0.500000,0.666667,1.000000,1,3,0.000379,0.250000,0,1,2,1,2,2,4,6513,0.500000,0.000417,0.605236,1.814534,0.000000,7.568889,0.000000,4.903333,0.556775,1.330467,0.697379,United-States,United-States,United-States,United-States,United-States,United-States,United-States,United-States,United-States,United-States,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [75]:
overall_fairness = fairness.get_overall_fairness(fairness_df)
print(overall_fairness)

{'Unsupervised Fairness': False, 'Supervised Fairness': False, 'Overall Fairness': False}
